In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [5]:
def getValidatorAndDelegationRewards():
    acc_rewards = dict()
    delegations = dict()


In [6]:
def R4_test():
    print("Test-R4: Reward given out to delegators sums up to the total delegation reward for each validator")
    num = int(input("enter the number of blocks you want to test: "))
    
    current_block = getBlockNumber()
    next_block = current_block + 1
    
    acc_rewards_prev = dict()
    delegations_prev = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            reward_accumulated = i['lifetime']['reward-accumulated']
            acc_rewards_prev[address] = reward_accumulated
            ds = i['validator']['delegations']
            dels = dict()
            for d in ds:
                d_addr = d['delegator-address']
                d_reward = d['reward']
                dels[d_addr] = d_reward
            delegations_prev[address] = dels  
            
    current_block = getBlockNumber()
    while current_block < next_block:
        current_block = getBlockNumber()
    
    iterations = 0
    while iterations < num:
        print("current block", current_block)
        # get the validator info and compute validator rewards
        acc_rewards_curr = dict()
        delegations_curr = dict()
        validator_infos = getAllValidatorInformation()
        for i in validator_infos:
            if i['currently-in-committee'] == True:
                address = i['validator']['address']
                reward_accumulated = i['lifetime']['reward-accumulated']
                acc_rewards_curr[address] = reward_accumulated
                if address not in acc_rewards_prev:
                    continue
                reward = reward_accumulated - acc_rewards_prev[address]
                if reward == 0:
                    continue
                elif reward < 0:
                    reward = -reward # first time delegations
                del_rewards = 0
                dels = delegations_prev[address]
                ds = i['validator']['delegations']
                for d in ds:
                    d_addr = d['delegator-address']
                    d_reward = d['reward']
                    del_rewards = del_rewards + d['reward']
                    if d_addr in dels:
                        del_rewards = del_rewards - dels[d_addr]
                if del_rewards != reward:
                    print("Test-R4 pass?", (del_rewards == reward), "for validator", address, \
                        "validator reward", reward, "delegators reward", del_rewards)
                delegations_curr[address] = ds
        
        last_block = current_block
        current_block = getBlockNumber()
        while current_block == last_block:
            current_block = getBlockNumber()
        
        acc_rewards_prev = acc_rewards_curr
        delegations_prev = delegations_curr
        
        iterations = iterations + 1
    print("done!")
    

In [7]:
R4_test()

Test-R4: Reward given out to delegators sums up to the total delegation reward for each validator
enter the number of blocks you want to test: 2
current block 2199
Test-R4 pass? False for validator one1marnnvc8hywmfxhrc8mtpjkvvdt32x9kxtwkvv validator reward 2245283018867926100 delegators reward 2245283018867926101
Test-R4 pass? False for validator one1j6yxw3elz4yvx3dku46uxynkvxd43n0eg0p3pf validator reward 2245283018867924532 delegators reward 2245283018867924533
current block 2200
Test-R4 pass? False for validator one1tpxl87y4g8ecsm6ceqay49qxyl5vs94jjyfvd9 validator reward 12150943396226415200 delegators reward 12601652594389567871566
Test-R4 pass? False for validator one1jl3p3huw0yzsnsnpdux3vjsy9v97rnxgp2gptu validator reward 3425531914893617032 delegators reward 3443703546325932942258
Test-R4 pass? False for validator one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh validator reward 2531914893617021248 delegators reward 3139201665911872100568
Test-R4 pass? False for validator one1marnnvc8